# 1. Gerekli Kütüphaneler

In [37]:
import os
import time
import threading
from datetime import datetime
from tkinter import Tk, Label, Button, Text, Scrollbar, Entry, filedialog, messagebox, END, RIGHT, Y, BOTH
from tkinter.filedialog import asksaveasfilename
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler


# 2. Log Yazma Fonksiyonu

In [40]:
def log_yaz(mesaj):
    zaman = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log = f"[{zaman}] {mesaj}"
    text_area.insert(END, log + "\n")
    text_area.see(END)
    with open("paylasim_log.txt", "a", encoding="utf-8") as f:
        f.write(log + "\n")


# 3. Dosya Olaylarını Takip Eden Handler Sınıfı (Filtreli)

In [43]:
class DosyaHandler(FileSystemEventHandler):
    def __init__(self, log_func, filtreli_uzantilar):
        self.log_func = log_func
        self.uzantilar = [u.strip().lower() for u in filtreli_uzantilar]

    def kontrol_et(self, path):
        return any(path.lower().endswith(f".{u}") for u in self.uzantilar)

    def on_created(self, event):
        if not event.is_directory and self.kontrol_et(event.src_path):
            self.log_func(f"🟢 Yeni dosya eklendi: {event.src_path}")

    def on_deleted(self, event):
        if not event.is_directory and self.kontrol_et(event.src_path):
            self.log_func(f"🔴 Dosya silindi: {event.src_path}")

    def on_modified(self, event):
        if not event.is_directory and self.kontrol_et(event.src_path):
            self.log_func(f"🟡 Dosya düzenlendi: {event.src_path}")

    def on_moved(self, event):
        if not event.is_directory and self.kontrol_et(event.src_path):
            self.log_func(f"🔵 Dosya yeniden adlandırıldı: {event.src_path} → {event.dest_path}")


# 4. Klasör Seçme Fonksiyonu

In [46]:
def klasor_sec():
    global izlenen_klasor
    izlenen_klasor = filedialog.askdirectory()
    status_label.config(text=f"İzlenen klasör: {izlenen_klasor}")


# 5. İzlemeyi Başlatma Fonksiyonu

In [49]:
def izlemeyi_baslat():
    global observer
    if not izlenen_klasor:
        status_label.config(text="⚠️ Önce bir klasör seçin.")
        return

    uzanti_str = uzanti_entry.get()
    filtreler = uzanti_str.split(",") if uzanti_str else []

    event_handler = DosyaHandler(log_yaz, filtreler)
    observer = Observer()
    observer.schedule(event_handler, izlenen_klasor, recursive=True)
    observer.start()
    status_label.config(text="✅ İzleme başlatıldı.")


# 6. İzlemeyi Durdurma Fonksiyonu

In [52]:
def izlemeyi_durdur():
    global observer
    if observer:
        observer.stop()
        observer.join()
        status_label.config(text="🛑 İzleme durduruldu.")


# 7. Logları Kaydetme Fonksiyonu

In [55]:
def log_kaydet():
    log_icerik = text_area.get("1.0", END).strip()
    if not log_icerik:
        messagebox.showinfo("Boş", "Kaydedilecek log bulunamadı.")
        return
    dosya_yolu = asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt")])
    if dosya_yolu:
        with open(dosya_yolu, "w", encoding="utf-8") as f:
            f.write(log_icerik)
        messagebox.showinfo("Başarılı", f"Log kaydedildi:\n{dosya_yolu}")


# 8. Log Temizleme Fonksiyonu

In [58]:
def log_temizle():
    cevap = messagebox.askyesno("Onay", "Tüm log ekranı temizlensin mi?")
    if cevap:
        text_area.delete("1.0", END)
        messagebox.showinfo("Temizlendi", "Log ekranı sıfırlandı.")


# 9. Tkinter Arayüzü (GUI)

In [63]:
pencere = Tk()
pencere.title("📁 FileOrbis Simülasyonu: Dosya Paylaşım İzleyici")
pencere.geometry("750x450")

status_label = Label(pencere, text="📂 İzlenecek klasörü seçin", fg="blue")
status_label.pack(pady=5)

Button(pencere, text="Klasör Seç", command=klasor_sec).pack(pady=2)
Button(pencere, text="İzlemeyi Başlat", command=lambda: threading.Thread(target=izlemeyi_baslat).start()).pack(pady=2)
Button(pencere, text="İzlemeyi Durdur", command=izlemeyi_durdur).pack(pady=2)
Button(pencere, text="Logları Kaydet", command=log_kaydet, bg="lightgray").pack(pady=2)
Button(pencere, text="Logları Temizle", command=log_temizle, bg="lightyellow").pack(pady=2)

Label(pencere, text="İzlenecek uzantılar (örn: pdf, docx):", fg="black").pack()
uzanti_entry = Entry(pencere, width=30)
uzanti_entry.insert(0, "pdf,docx")  # varsayılan örnek uzantılar
uzanti_entry.pack(pady=2)

scrollbar = Scrollbar(pencere)
scrollbar.pack(side=RIGHT, fill=Y)

text_area = Text(pencere, yscrollcommand=scrollbar.set)
text_area.pack(fill=BOTH, expand=True)
scrollbar.config(command=text_area.yview)

izlenen_klasor = ""
observer = None

pencere.mainloop()
